In [7]:
import os
import sys
import json
import subprocess
import shutil
import random
import re
import math
import carla
import csv

if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import importlib
from pprint import pprint
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np

# Add the "scripts" directory to sys.path
current_dir = os.path.dirname(os.path.abspath("__file__"))
scripts_dir = os.path.abspath(os.path.join(current_dir, 'scripts'))
sys.path.append(scripts_dir)
import vehParameters
import osmAPI as osm
import LLAMAconnect

FOLDER_NAME = "town01"

In [12]:
NET_PATH = f'{FOLDER_NAME}/osm.net.xml'
ADDITONALS_PATH = f'{FOLDER_NAME}/park.add.xml'
SUMOCFG_PATH = f'{FOLDER_NAME}/osm.sumocfg'
PA_REROUTER_PATH = f'{FOLDER_NAME}/pa_rerouter.xml'

net = sumolib.net.readNet(NET_PATH)
parkingAreas = list(sumolib.output.parse(ADDITONALS_PATH, "parkingArea"))

In [13]:
net.getLocationOffset()

[0.06, 328.61]

In [ ]:
def getClosestEdges(lat, lon, radius, maxEdges=10, convert_toXY=True):

    # Gets the 10 closest edges to the given lat, lon
    if convert_toXY:
        x, y = net.convertLonLat2XY(lon, lat)
    else:
        x, y = lon, lat
    edges = net.getNeighboringEdges(x, y, radius)
    closestEdges = []
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])

        ## Checking if the edge found can be used by passenger car
        for dist, edge in distanceAndEdges:
            if edge.allows('passenger'):
                closestEdges.append(edge)

    if len(edges) == 0:
        print(f'No edges found for {lat}, {lon}. Perhaps location is not inside the network or there are no viable roads inside the radius.')
        return None
    
    return closestEdges

In [5]:
def get_closest_spawn_point(transform, spawn_points):
    closest_point = min(spawn_points, key=lambda sp: sp.location.distance(transform.location))
    return closest_point

In [9]:
def get_spawn_points_from_csv(csv_file):
    spawn_points = []
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            loc = carla.Location(float(row['x']), float(row['y']), float(row['z']))
            rot = carla.Rotation(float(row['pitch']), float(row['yaw']), float(row['roll']))
            spawn_points.append(carla.Transform(loc, rot))
    
    return spawn_points

In [15]:
sps = get_spawn_points_from_csv('interests_spawn_points_Town01.csv')
print(sps)